## 量子回路学習

まず、必要なライブラリを最初にインポートします。

In [ ]:
import os
import sys
import shutil
import tarfile
from google.colab import drive

drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/UTokyo-ICEPP/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/

In [1]:
%pip install qiskit-machine-learning

# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import TwoLocal, ZFeatureMap, ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator, Sampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import SLSQP, COBYLA
from qiskit_ibm_runtime import Session, Sampler as RuntimeSampler
from qiskit_ibm_runtime.accounts import AccountNotFoundError
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms import NeuralNetworkRegressor

## 初歩的な例
ある入力$\{x_i\}$と、既知の関数$f$による出力$y_i=f(x_i)$が学習データとして与えられた時に、そのデータから関数$f$を近似的に求める問題を考えてみます。例として、$f(x)=x^3$としてみます。

### 学習データの準備

In [ ]:
random_seed = 0
rng = np.random.default_rng(random_seed)

# Qubit数、変分フォームのレイヤー数、訓練サンプル数の定義など
nqubit = 3
nlayer = 5
x_min = -1.
x_max = 1.
num_x_train = 30
num_x_validation = 20

# 関数の定義
func_to_learn = lambda x: x ** 3

# 学習用データセットの生成
x_train = rng.uniform(x_min, x_max, size=num_x_train)
y_train = func_to_learn(x_train)

# 関数に正規分布ノイズを付加
mag_noise = 0.05
y_train_noise = y_train + rng.normal(0., mag_noise, size=num_x_train)

# 検証用データセットの生成
x_validation = rng.uniform(x_min, x_max, size=num_x_validation)
y_validation = func_to_learn(x_validation) + rng.normal(0., mag_noise, size=num_x_validation)

# 学習用データをプロットして確認
x_list = np.arange(x_min, x_max, 0.02)
plt.plot(x_train, y_train_noise, "o", label='Training Data (w/ Noise)')
plt.plot(x_list, func_to_learn(x_list), label='Original Function')
plt.legend()

### 量子状態の生成

In [ ]:
u_in = QuantumCircuit(nqubit, name='U_in')
x = Parameter('x')

for iq in range(nqubit):
    # parameter.arcsin()はparameterに値vが代入された時にarcsin(v)になるパラメータ表現
    u_in.ry(x.arcsin(), iq)
    # arccosも同様
    u_in.rz((x * x).arccos(), iq)

u_in.assign_parameters({x: x_train[0]}).draw('mpl')

### 変分フォームを使った状態変換

In [ ]:
u_out = QuantumCircuit(nqubit, name='U_out')

# 長さ0のパラメータ配列
theta = ParameterVector('θ', 0)

# thetaに一つ要素を追加して最後のパラメータを返す関数
def new_theta():
    theta.resize(len(theta) + 1)
    return theta[-1]

for iq in range(nqubit):
    u_out.ry(new_theta(), iq)

for iq in range(nqubit):
    u_out.rz(new_theta(), iq)

for iq in range(nqubit):
    u_out.ry(new_theta(), iq)

for il in range(nlayer):
    for iq in range(nqubit):
        u_out.cz(iq, (iq + 1) % nqubit)

    for iq in range(nqubit):
        u_out.ry(new_theta(), iq)

    for iq in range(nqubit):
        u_out.rz(new_theta(), iq)

    for iq in range(nqubit):
        u_out.ry(new_theta(), iq)

print(f'{len(theta)} parameters')

theta_vals = rng.uniform(0., 2. * np.pi, size=len(theta))

u_out.assign_parameters(dict(zip(theta, theta_vals))).draw('mpl')

### 測定とモデル出力

In [ ]:
model = QuantumCircuit(nqubit, name='model')

model.compose(u_in, inplace=True)
model.compose(u_out, inplace=True)

assign_params = dict(zip(theta, theta_vals))
assign_params[x] = x_train[0]

model.assign_parameters(assign_params).draw('mpl')

In [8]:
# 今回はバックエンドを利用しない（量子回路シミュレーションを簡略化した）Estimatorクラスを使う
#estimator = Estimator()
backend = AerSimulator()
estimator = BackendEstimator(backend)

# 与えられたパラメータの値とxの値に対してyの値を計算する
def yvals(param_vals, x_vals=x_train):
    circuits = list()
    for x_val in x_vals:
        # xだけ数値が代入された変分回路
        circuits.append(model.assign_parameters({x: x_val}))

    # 観測量はIIZ（右端が第0量子ビット）
    observable = SparsePauliOp('I' * (nqubit - 1) + 'Z')

    # shotsは関数の外で定義
    job = estimator.run(circuits, [observable] * len(circuits), [param_vals] * len(circuits), shots=shots)

    return np.array(job.result().values)

def objective_function(param_vals):
    return np.sum(np.square(y_train_noise - yvals(param_vals)))

def callback_function(param_vals):
    # lossesは関数の外で定義
    losses.append(objective_function(param_vals))

    if len(losses) % 10 == 0:
        print(f'COBYLA iteration {len(losses)}: cost={losses[-1]}')

In [9]:
# COBYLAの最大ステップ数
maxiter = 100
# COBYLAの収束条件（小さいほどよい近似を目指す）
tol = 0.01
# バックエンドでのショット数
shots = 1000

optimizer = COBYLA(maxiter=maxiter, tol=tol, callback=callback_function)

In [ ]:
initial_params = rng.uniform(0., 2. * np.pi, size=len(theta))

losses = list()
min_result = optimizer.minimize(objective_function, initial_params)
plt.plot(losses)

In [ ]:
x_list = np.linspace(x_min, x_max, 100)

y_pred = yvals(min_result.x, x_vals=x_list)

# 結果を図示する
plt.plot(x_train, y_train_noise, "o", label='Training Data (w/ Noise)')
plt.plot(x_list, func_to_learn(x_list), label='Original Function')
plt.plot(x_list, np.array(y_pred), label='Predicted Function')
plt.legend();

## 素粒子現象の探索への応用

### 学習データの準備

In [ ]:
# ファイルから変数を読み出す
input_file = os.getcwd()+'/qc-workbook-lecturenotes/data/SUSY_1K.csv'
df = pd.read_csv(input_file,
                 names=('isSignal','lep1_pt','lep1_eta','lep1_phi','lep2_pt','lep2_eta',
                        'lep2_phi','miss_ene','miss_phi','MET_rel','axial_MET','M_R','M_TR_2',
                        'R','MT2','S_R','M_Delta_R','dPhi_r_b','cos_theta_r1'))

# 学習に使う変数の数
feature_dim = 3  # dimension of each data point

# 3, 5, 7変数の場合に使う変数のセット
if feature_dim == 3:
    selected_features = ['lep1_pt', 'lep2_pt', 'miss_ene']
elif feature_dim == 5:
    selected_features = ['lep1_pt','lep2_pt','miss_ene','M_TR_2','M_Delta_R']
elif feature_dim == 7:
    selected_features = ['lep1_pt','lep1_eta','lep2_pt','lep2_eta','miss_ene','M_TR_2','M_Delta_R']

# 学習に使う事象数: trainingは訓練用サンプル、testingはテスト用サンプル
train_size = 20
test_size = 20

df_sig = df.loc[df.isSignal==1, selected_features]
df_bkg = df.loc[df.isSignal==0, selected_features]

# サンプルの生成
df_sig_train = df_sig.values[:train_size]
df_bkg_train = df_bkg.values[:train_size]
df_sig_test = df_sig.values[train_size:train_size + test_size]
df_bkg_test = df_bkg.values[train_size:train_size + test_size]
# 最初のtrain_size事象がSUSY粒子を含む信号事象、残りのtrain_size事象がSUSY粒子を含まない背景事象
train_data = np.concatenate([df_sig_train, df_bkg_train])
# 最初のtest_size事象がSUSY粒子を含む信号事象、残りのtest_size事象がSUSY粒子を含まない背景事象
test_data = np.concatenate([df_sig_test, df_bkg_test])

# ラベル
train_label = np.zeros(train_size * 2, dtype=int)
train_label[:train_size] = 1
test_label = np.zeros(train_size * 2, dtype=int)
test_label[:test_size] = 1

# one-hotベクトル（信号事象では第1次元の第0要素が1、背景事象では第1次元の第1要素が1）
train_label_one_hot = np.zeros((train_size * 2, 2))
train_label_one_hot[:train_size, 0] = 1
train_label_one_hot[train_size:, 1] = 1
test_label_one_hot = np.zeros((test_size * 2, 2))
test_label_one_hot[:test_size, 0] = 1
test_label_one_hot[test_size:, 1] = 1

#datapoints, class_to_label = split_dataset_to_data_and_labels(test_input)
#datapoints_tr, class_to_label_tr = split_dataset_to_data_and_labels(training_input)

mms = MinMaxScaler((-1, 1))
norm_train_data = mms.fit_transform(train_data)
norm_test_data = mms.transform(test_data)

## 量子ニューラルネットワーク回路

### 量子状態の生成

In [ ]:
#feature_map = ZFeatureMap(feature_dimension=feature_dim, reps=1)
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1, entanglement='circular')
feature_map.decompose().draw('mpl')

### 変分フォームを使った状態変換

In [ ]:
ansatz = TwoLocal(num_qubits=feature_dim, rotation_blocks=['ry', 'rz'], entanglement_blocks='cz', entanglement='circular', reps=3)
#ansatz = TwoLocal(num_qubits=feature_dim, rotation_blocks=['ry'], entanglement_blocks='cz', entanglement='circular', reps=3)
ansatz.decompose().draw('mpl')

### 測定とモデル出力

In [15]:
# 上のEstimatorと同じく、バックエンドを使わずシミュレーションを簡略化したSampler
sampler = Sampler()

# 実機で実行する場合
# instance = 'ibm-q/open/main'

# try:
#     service = QiskitRuntimeService(channel='ibm_quantum', instance=instance)
# except AccountNotFoundError:
#     service = QiskitRuntimeService(channel='ibm_quantum', token='__paste_your_token_here__',
#                                    instance=instance)

# backend_name = 'ibm_washington'
# session = Session(service=service, backend=backend_name)

# sampler = RuntimeSampler(session=session)

maxiter = 300

optimizer = COBYLA(maxiter=maxiter, disp=True)

objective_func_vals = []
# Draw the value of objective function every time when the fit() method is called
def callback_graph(weights, obj_func_eval):
    #clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    if len(objective_func_vals) % 10 == 0:
        print(f'COBYLA iteration {len(objective_func_vals)}: cost={obj_func_eval}')

    #plt.title("Objective function value against iteration")
    #plt.xlabel("Iteration")
    #plt.ylabel("Objective function value")
    #plt.plot(objective_func_vals)
    #plt.show()

vqc = VQC(num_qubits=feature_dim,
          feature_map=feature_map,
          ansatz=ansatz,
          loss="cross_entropy",
          optimizer=optimizer,
          callback=callback_graph,
          sampler=sampler)

### シミュレーションで実行

In [ ]:
vqc.fit(norm_train_data, train_label_one_hot)

train_score = vqc.score(norm_train_data, train_label_one_hot)
test_score = vqc.score(norm_test_data, test_label_one_hot)

print(f'--- Classification Train score: {train_score} ---')
print(f'--- Classification Test score:  {test_score} ---')

## 量子カーネル

### 特徴量マップ

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1, entanglement='circular')
feature_map.decompose().draw('mpl')

### フィデリティカーネル

In [ ]:
# FidelityQuantumKernel creates a Sampler instance internally
q_kernel = FidelityQuantumKernel(feature_map=feature_map)

### 量子カーネル計算用の回路

In [ ]:
bind_params = dict(zip(feature_map.parameters, norm_train_data[0]))
feature_map_0 = feature_map.assign_parameters(bind_params)
bind_params = dict(zip(feature_map.parameters, norm_train_data[1]))
feature_map_1 = feature_map.assign_parameters(bind_params)

qc_circuit = q_kernel.fidelity.create_fidelity_circuit(feature_map_0, feature_map_1)
qc_circuit.decompose().decompose().draw('mpl')

###  10000回の測定でフィデリティカーネルを計算

In [ ]:
sampler = Sampler()

job = sampler.run(qc_circuit, shots=10000)

# quasi_dists[0] is the probability distribution of expected measured counts
fidelity = job.result().quasi_dists[0].get(0, 0.)
print(f'|<φ(x_1)|φ(x_0)>|^2 = {fidelity}')

### カーネル行列を表示

In [ ]:
matrix_train = q_kernel.evaluate(x_vec=norm_train_data)
matrix_test = q_kernel.evaluate(x_vec=norm_test_data, y_vec=norm_train_data)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train), interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test), interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("validation kernel matrix")
plt.show()

### サポートベクターマシンでデータ分類

In [ ]:
qc_svc = SVC(kernel='precomputed') # Default value of hyperparameter (C) is 1
qc_svc.fit(matrix_train, train_label)

train_score = qc_svc.score(matrix_train, train_label)
test_score = qc_svc.score(matrix_test, test_label)

print(f'Precomputed kernel: Classification Train score: {train_score*100}%')
print(f'Precomputed kernel: Classification Test score:  {test_score*100}%')

## 量子データを使った量子機械学習

イジング模型の設定は5/23の授業で使ったものと同じです。

まずRyと制御Zゲートからなる2量子ビット回路を考えます。

In [ ]:
# VQE setup
num_qubits = 2

ansatz = TwoLocal(num_qubits, "ry", "cz", reps=3)  # Ry gates with trainable parameters and CZ for entanglement
ansatz.decompose().draw('mpl')

ここで考えるイジング模型には横磁場と縦磁場の混合度合いを表すパラメータ$\alpha$が入っていて、$\alpha = 0$が純粋な横磁場、$\alpha = \pi/2$が純粋な縦磁場のモデルに対応しています。


In [ ]:
# Estimatorを使う
estimator = Estimator()
optimizer = SLSQP(maxiter=1000)  # Classical optimizer

# QiskitのVQEクラスを使う
vqe = VQE(estimator, ansatz, optimizer)

# 横磁場と縦磁場を持つイジング模型のハミルトニアンを定義
def get_hamiltonian(L, J, h, alpha=0):

    # タプル型の変数として、ハミルトニアンの各項からなるリストを定義
    # (1) パウリ列
    # (2) パウリ列に対応する量子ビットのインデックス
    # (3) パウリ列の係数
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * np.sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * np.cos(alpha)) for i in range(0, L)]

    # `from_sparse_list`を使い、SparsePauliOpとしてハミルトニアンを作る
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

 例として、$\alpha = \pi/8$として、縦磁場と横磁場が混ざっているケースでの基底エネルギーをVQEで求めてみます。隣接ビット間の結合定数は$J = 0.2$、磁場との結合定数は$h = 1.2$とします。


In [ ]:
# パラメータの設定
J = 0.2
h = 1.2
alpha = np.pi/8
H = get_hamiltonian(L=num_qubits, J=J, h=h, alpha=alpha)

# VQEで最低エネルギー状態を求める
result = vqe.compute_minimum_eigenvalue(H)
#print(result)
print(f'VQE energy value = {result.optimal_value:.5f}')

系のサイズが小さいので、厳密対角化して最低エネルギーを計算することが可能です。


In [ ]:
# ハミルトニアンを厳密体格化し、基底エネルギーを求める
numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=H)
ref_value = result.eigenvalue.real
print(f"Reference energy value = {ref_value:.5f}")

## 基底状態からハミルトニアンパラメータを決定する

まず、異なる$h$の値に対応する基底状態のデータセットをVQEを使って生成します。単純化のため、`alpha`パラメータは0とします。

In [ ]:
Nexp = 5   # あるhの値に対して行う実験回数
Ntrain = 20  # 実験あたりの訓練データの数 → この数だけhの値を変えたデータセットを作る

J = 0.2
alpha = 0

# hの値を[0, 1]の範囲でランダムに選ぶ
h_list = [np.random.rand() for _ in range(Ntrain)]
print(f'Input field strenghs = {h_list}')

上で使ったのと同じ一般的な量子回路（`TwoLocal`）を使ってVQEを行い、得られたエネルギーの値をチェックしてみます。


In [ ]:
# 一般的なアンザッツを使ってVQEを行う
vqe_ansatz = TwoLocal(num_qubits, "ry", "cz", parameter_prefix='x')

# OptimizerはSLSQPとする
optimizer = SLSQP(maxiter=1000)
estimator = Estimator()
vqe = VQE(estimator, vqe_ansatz, optimizer)

# 異なるhの値を持つ訓練データに対してVQEを行う
opt_vqe_energy = []
opt_vqe_params = []
for i in range(Ntrain):
    H = get_hamiltonian(L=num_qubits, J=J, h=h_list[i], alpha=alpha)
    result_vqe = vqe.compute_minimum_eigenvalue(H)
    opt_vqe_energy.append(result_vqe.optimal_value)
    opt_vqe_params.append(list(result_vqe.optimal_parameters.values()))
    print('VQE i =',i)

# エネルギー値を表示
for i in range(Ntrain):
    print(f'VQE[{i}] energy value = {opt_vqe_energy[i]:.5f}')

厳密対角化して得られたエネルギーと比較して、VQEの結果を確認


In [ ]:
numpy_solver = NumPyMinimumEigensolver()
for i in range(Ntrain):
    H = get_hamiltonian(L=num_qubits, J=J, h=h_list[i], alpha=alpha)
    result = numpy_solver.compute_minimum_eigenvalue(operator=H)
    ref_value = result.eigenvalue.real
    print(f"Reference[{i}] energy value = {ref_value:.5f}")

ここで量子機械学習用の回路を作り、ハミルトニアンパラメータの決定を行います。最初に最適化したVQEの回路（`vqe_ansatz`）を置いて基底状態を作り、その後に学習用の量子回路を置いてパラメータ決定をすることを考えます。これはVQE回路を特徴量マップとしてみなすやり方で、こうすることで最適化したVQE回路のパラメータを入力データのように扱うことができます。


In [ ]:
# 最初の量子ビットを計算基底で測定する
pauli = 'I' * (num_qubits - 1)
pauli += 'Z'
obs = SparsePauliOp([pauli], coeffs = 10.)

# 損失関数の値を表示するCallback関数
def callback_graph(weights, obj_func_eval):
    objective_func_vals.append(obj_func_eval)

# 学習用量子回路のCX-RYレイヤーの数
nlayer = 3  # CX-RY layers

result_exp = []
regressor_exp = []
objective_func_vals_exp = []

# 実験をNexp回繰り返す
for iexp in range(Nexp):
    
    qc = QuantumCircuit(num_qubits)
    
    # VQE回路
    qc.compose(vqe_ansatz, inplace=True)

    # 学習用の量子回路
    qnn_ansatz = RealAmplitudes(num_qubits=num_qubits, reps=nlayer, parameter_prefix='theta')
    qc.compose(qnn_ansatz, inplace=True)

    # 学習パラメータの初期値：[0, pi]でランダムに設定
    initial_weights = np.random.rand((nlayer+1)*num_qubits)*np.pi

    # EstimatorQNNクラスを使う
    qnn = EstimatorQNN(
        circuit = qc,
        input_params = vqe_ansatz.parameters,
        weight_params = qnn_ansatz.parameters,
        observables = obs
    )

    # NeuralNetworkRegressorクラスを使い、回帰問題としてhの値を予測する
    regressor = NeuralNetworkRegressor(
        neural_network = qnn,
        loss = "squared_error",
        optimizer = SLSQP(maxiter=1000),
        warm_start = True,
        initial_point = initial_weights,
        callback = callback_graph
    )

    objective_func_vals = []
    # 最適化したVQE回路パラメータを入力データ、上で設定したhの値を出力データとして回帰を行う
    result_regres = regressor.fit(np.array(opt_vqe_params),np.array(h_list))
    result_exp.append(result_regres)
    regressor_exp.append(regressor)
    objective_func_vals_exp.append(objective_func_vals)

    print(f'iexp = {iexp}')

損失関数のプロファイルを表示する。


In [ ]:
fig = plt.figure(facecolor="w")
ax = fig.add_subplot(1, 1, 1)
plt.title('Objective function value against iteration')
plt.xlabel("Iteration")
plt.ylabel("Objective function value")
for iexp in range(Nexp):
    plt.plot(range(len(objective_func_vals_exp[iexp])), objective_func_vals_exp[iexp])
plt.show()

設定した$h$の値（横軸）と予測した$h$の値（縦軸）をプロットする。


In [ ]:
train_pred_exp = []
for iexp in range(Nexp):
    train_pred = regressor_exp[iexp].predict(np.array(opt_vqe_params))
    train_pred_exp.append(train_pred)
    plt.scatter(h_list, train_pred, label='training')
plt.title('True vs Predicted values')
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.xlim(-0.05,1.1)
plt.ylim(-0.15,1.25)
plt.plot([-0.2,1.2],[-0.2,1.2],'k--')
plt.show()